In [1]:
import subprocess
import os

import pandas as pd
import numpy as np
import requests

from datetime import datetime

from bs4 import BeautifulSoup


%matplotlib inline


import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns

import plotly.graph_objects as go

In [2]:
mpl.rcParams['figure.figsize'] = [16, 9]
pd.set_option('display.max_rows', 500)

sns.set(style="darkgrid")

![CRISP_DM](../reports/figures/CRISP_DM.png)

# DATA UNDERSTANDING

RKI, webscrape (webscraping) https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html

John Hopkins (GITHUB) https://github.com/CSSEGISandData/COVID-19.git

REST API services to retreive data https://npgeo-corona-npgeo-de.hub.arcgis.com/

# Git cloaning

In [23]:
data_path= '../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
pd_raw=pd.read_csv(data_path)

In [24]:
pd_raw.tail()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/4/20,9/5/20,9/6/20,9/7/20,9/8/20,9/9/20,9/10/20,9/11/20,9/12/20,9/13/20
261,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,...,25142,25575,26127,26779,27363,27919,28664,29256,29906,30574
262,NaN,Western Sahara,24.215500,-12.885800,0,0,0,0,0,0,...,10,10,10,10,10,10,10,10,10,10
263,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,1983,1983,1987,1989,1994,1999,2003,2007,2009,2011
264,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,...,12639,12709,12776,12836,12952,13112,13214,13323,13466,13539
265,NaN,Zimbabwe,-19.015438,29.154857,0,0,0,0,0,0,...,6837,6837,6837,7298,7388,7429,7453,7479,7508,7526


# WEB SCRAPING

In [5]:
page=requests.get("https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html")

In [6]:
soup=BeautifulSoup(page.content,'html.parser')

In [7]:
html_table=soup.find('table')

In [8]:
all_rows=html_table.find_all('tr')

In [9]:
final_data_list=[]

In [10]:
for pos,rows in enumerate(all_rows):
    col_list=[each_col.get_text(strip=True) for each_col in rows.find_all('td')] 
    final_data_list.append(col_list)

In [11]:
pd_daily_status=pd.DataFrame(final_data_list).dropna().rename(columns={0:'state',
                                                                       1:'cases',
                                                                       2:'chnages',
                                                                       3:'cases_per_100k',
                                                                       4:'fatal',
                                                                       5:'comment'})

In [12]:
pd_daily_status.head()

,state,cases,chnages,cases_per_100k,fatal,comment
2,Baden-Württem­berg,45.254,+184,1.423,"12,9",1.867
3,Bayern,61.974,+269,2.328,"17,8",2.645
4,Berlin,12.269,+15,532,"14,2",226
5,Branden­burg,4.006,+1,65,"2,6",169
6,Bremen,2.135,+5,65,"9,5",58


# REST API

In [13]:
data=requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/Coronaf%C3%A4lle_in_den_Bundesl%C3%A4ndern/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')

In [14]:
import json

In [15]:
json_object=json.loads(data.content)

In [16]:
type(json_object)

dict

In [17]:
json_object.keys()

dict_keys(['objectIdFieldName', 'uniqueIdField', 'globalIdFieldName', 'geometryProperties', 'serverGens', 'geometryType', 'spatialReference', 'fields', 'features'])

In [18]:
full_list=[]
for pos,each_dict in enumerate (json_object['features'][:]):
    full_list.append(each_dict['attributes'])

In [19]:
pd.DataFrame(full_list)

,OBJECTID_1,LAN_ew_AGS,LAN_ew_GEN,LAN_ew_BEZ,LAN_ew_EWZ,OBJECTID,Fallzahl,Aktualisierung,AGS_TXT,GlobalID,faelle_100000_EW,Shape__Area,Shape__Length,Death
0,1,01,Schleswig-Holstein,Land,2896712,15,4231,1600034400000,01,fc5ba936-c95c-432c-8a33-9eb2f30b660f,146.062156,4.573731e+10,2.881496e+06,161
1,2,02,Hamburg,Freie und Hansestadt,1841179,6,6825,1600034400000,02,0f3e860c-5181-4d3f-a421-1d51f50315ea,370.686392,2.089396e+09,4.188002e+05,268
2,3,03,Niedersachsen,Land,7982448,9,17911,1600034400000,03,3fd77024-c29b-4843-9be8-682ad48e60c9,224.379789,1.299836e+11,4.008988e+06,667
3,4,04,Bremen,Freie Hansestadt,682986,5,2135,1600034400000,04,4132268b-54de-4327-ac1e-760e915112f1,312.597916,1.119157e+09,3.357177e+05,58
4,5,05,Nordrhein-Westfalen,Land,17932651,10,62261,1600034400000,05,561d658f-3ee5-46e3-bc95-3528c6558ab9,347.193508,8.782936e+10,2.648673e+06,1829
5,6,06,Hessen,Land,6265809,7,16877,1600034400000,06,93277ac4-e8fc-48c7-8940-028dc2ed66af,269.350694,5.235913e+10,2.148244e+06,539
6,7,07,Rheinland-Pfalz,Land,4084844,11,9764,1600034400000,07,e9b4296f-9be2-4e53-9a58-ccf1396cb03d,239.029936,4.783877e+10,1.774430e+06,246
7,8,08,Baden-Württemberg,Land,11069533,1,45254,1600034400000,08,80394ddf-c6a4-4a6e-be8e-0259a81b22a9,408.815801,8.151732e+10,2.544320e+06,1867
8,9,09,Bayern,Freistaat,13076721,2,61974,1600034400000,09,1ff920f4-62cd-4a4f-b8c9-f042f2a3e00a,473.926147,1.634855e+11,3.898618e+06,2645
9,10,10,Saarland,Land,990509,12,3245,1600034400000,10,e3396a6f-8a30-4fdf-8df7-def77dd38bea,327.609340,6.060692e+09,5.626789e+05,175
